## Using SafeDrone for collision risk assessment

SafeDrone can be used for risk of collision of the drones. It takes the trajectory of the drones as an input, and calculates the risk of collision. 

The drone consists of three main electrical components that are essential for its operation.
1. Motor (Rotor)
2. Battery
3. Onboard chip

In the following, we show an example usage of drone collision risk assessment. 

In [1]:
import sys
import os

import inspect

from IPython.display import Markdown as md

from sys import platform
import pandas as pd
import numpy as np
import random

if platform == "linux" or platform == "linux2":
    currentdir = os.path.dirname(os.path.realpath(inspect.getfile(inspect.currentframe())))
    parentdir = os.path.dirname(currentdir)
    sys.path.insert(0, parentdir)
elif platform == "win32":
    # in Windows, the first path entry contains the directory where the notebook is
    sys.path.insert(0, os.path.dirname(sys.path[0]))

from SafeDrones.core.SafeDrones import SafeDrones

In [2]:
eval = SafeDrones()

Initialize the variables . . . .
Use $5$ as collision threshold, and $10$ as danger threshold. GPS lambda can be used the default ($0.001$).

In [3]:
eval.Set_Variables(collision_threshold=5, danger_threshold=10, GPS_Lambda=0.001)

Sample trajectories . . . . 

In [4]:
uav_1_trajectory = [(x, 0, 0) for x in range(10)]
uav_2_trajectory = [(x, 0, x) for x in range(10)]

danger_zone_risk, collision_zone_risk = eval.calculate_collision_risk(uav_1_trajectory, uav_2_trajectory)

print(danger_zone_risk)
print(collision_zone_risk)

0.5
0.5


In [5]:
danger_threshold = 5
collision_threshold = 2

danger_zone_risk, collision_zone_risk = eval.calculate_collision_risk(uav_1_trajectory, uav_2_trajectory, danger_threshold, collision_threshold)

print(danger_zone_risk)
print(collision_zone_risk)

0.3
0.2


To check the risk of GPS failure, let us first set the satellite status. The satellite status is a number between 0 to 18. Let us randomly assign the the status . . . .

In [6]:
sat_status = random.randint(0, 18)
eval.Set_Variables(SatStatus = sat_status)
print("The satellite status is set as : ", sat_status)

The satellite status is set as :  8


In [7]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(time=100)
print(GPS_P_Fail)
print(GPS_MTTF)

0.172620318489205
153.846153846154


In [8]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(SatStatus=14, time=100, Lambda = 0.001)
print(GPS_P_Fail)
print(GPS_MTTF)

1.19038762535297e-5
1142.85714285714


In [9]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(SatStatus=10, time=100, Lambda = 0.001)
print(GPS_P_Fail)
print(GPS_MTTF)

0.0107555729878382
363.636363636364


In [10]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(SatStatus=0, time=100, Lambda = 0.001)
print(GPS_P_Fail)
print(GPS_MTTF)

1
0
